In [29]:
import numpy as np

A = np.arange(24).reshape(2, 3, 4)
A[0], A.sum(axis=0)

(array([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]),
 array([[12, 14, 16, 18],
        [20, 22, 24, 26],
        [28, 30, 32, 34]]))

In [30]:
# np.pad
np.pad(A, [(0, 0), (2, 1), (1, 3)], mode="constant")  # pad的第二参数传入填充数量(前，后)的控制数组


array([[[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  1,  2,  3,  0,  0,  0],
        [ 0,  4,  5,  6,  7,  0,  0,  0],
        [ 0,  8,  9, 10, 11,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0]],

       [[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 12, 13, 14, 15,  0,  0,  0],
        [ 0, 16, 17, 18, 19,  0,  0,  0],
        [ 0, 20, 21, 22, 23,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0]]])

In [31]:
# np.transpose

A, A.transpose((0, 2, 1))

(array([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],
 
        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]),
 array([[[ 0,  4,  8],
         [ 1,  5,  9],
         [ 2,  6, 10],
         [ 3,  7, 11]],
 
        [[12, 16, 20],
         [13, 17, 21],
         [14, 18, 22],
         [15, 19, 23]]]))

In [32]:
# im2col - 理解测试
import sys, os
sys.path.append(os.path.pardir)
from common.util import im2col
x1 = np.random.rand(1, 3, 7, 7)
x2 = np.random.rand(10, 3, 7, 7)
x1.shape, x2.shape

((1, 3, 7, 7), (10, 3, 7, 7))

In [33]:
col1 = im2col(x1, 5, 5, stride=1, pad=0)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
col1.shape, col2.shape

((9, 75), (90, 75))

In [34]:
# im2col - 参数设置
input_data = x1
filter_h,filter_w = 5, 5
stride, pad = 1, 0

N, C, H, W = input_data.shape
out_h = (H + 2*pad - filter_h)//stride + 1
out_w = (W + 2*pad - filter_w)//stride + 1

img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

for y in range(filter_h):
    y_max = y + stride*out_h
    for x in range(filter_w):
        x_max = x + stride*out_w
        col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]   # 对每个数据、通道，收集索引为(y, x)整个需要相乘的数据点
        print(col[:, :, y, x, :, :])                                        # 例如，这里 10 x 3 x 3 x 3, 后面的3x3个数据点是5x5滤波器中的某一点在整个数据集中遍历的点

col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)            # 每一行代表3x5x5的滤波器变成列与之相乘的数据点, 每3x3行代表一个数据集
#                   数量,   通道, 滤波器长, 滤波器宽,  输出长,  输出宽 -> transpose
#                   数量, 输出长,   输出宽,    通道, 滤波器长, 滤波器宽
# 提示 reshape的顺序是按照0, 0, 0, 0, 0 - > 0, 0, 0, 0, 1 ... -> 0, 0, 0, 1, 0，最后一个轴是变化先变化，依次往前递进

[[[[0.27690048 0.76475499 0.68128395]
   [0.54320895 0.27959686 0.14511859]
   [0.64084363 0.08025998 0.21926138]]

  [[0.55615633 0.60518473 0.4631491 ]
   [0.15524087 0.97385814 0.43232387]
   [0.31289063 0.90241461 0.12008861]]

  [[0.98854492 0.59147446 0.79659866]
   [0.72561051 0.64966604 0.69502992]
   [0.22661769 0.41165645 0.89748373]]]]
[[[[0.76475499 0.68128395 0.9759702 ]
   [0.27959686 0.14511859 0.25309581]
   [0.08025998 0.21926138 0.97625881]]

  [[0.60518473 0.4631491  0.56542241]
   [0.97385814 0.43232387 0.54145095]
   [0.90241461 0.12008861 0.11256115]]

  [[0.59147446 0.79659866 0.28401414]
   [0.64966604 0.69502992 0.31786805]
   [0.41165645 0.89748373 0.70215884]]]]
[[[[0.68128395 0.9759702  0.27615381]
   [0.14511859 0.25309581 0.68592993]
   [0.21926138 0.97625881 0.37229464]]

  [[0.4631491  0.56542241 0.92087749]
   [0.43232387 0.54145095 0.92538702]
   [0.12008861 0.11256115 0.25345079]]

  [[0.79659866 0.28401414 0.13216539]
   [0.69502992 0.31786805 0.2854

In [35]:
from SimpleConvNet import im2col
# 验证自己写的im2col正确性
np.all(im2col(input_data, 5, 5, 1, 0) == col)

True

In [36]:
# np.zeros(num)
num = 3
np.zeros(num)

array([0., 0., 0.])

In [37]:
# 广播机制
W = A
b = np.array([1,2,3,4])
W, b, W+b   # 最后的n个轴匹配大小才可以进行广播，顺序是从后往前进行广播
            # b的大小: (4, )，W的大小: (2, 3, 4)，广播顺序b: (4, ) -> (3, 4) -> (2, 3, 4)

(array([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],
 
        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]),
 array([1, 2, 3, 4]),
 array([[[ 1,  3,  5,  7],
         [ 5,  7,  9, 11],
         [ 9, 11, 13, 15]],
 
        [[13, 15, 17, 19],
         [17, 19, 21, 23],
         [21, 23, 25, 27]]]))

In [38]:
# np.size

A.size

24

In [39]:
# tuple相加

(1, 2) + (3,)

(1, 2, 3)

In [40]:
# flatten

A, A.flatten(), A.reshape(-1)==A.flatten()

(array([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],
 
        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23]),
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True]))

In [41]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist_origin
from SimpleConvNet import SimpleConvNet
# 数据读入
(x_train, t_train), (x_test, t_test) = load_mnist_origin(one_hot_lable=True)
x_train.shape, x_test.shape

((60000, 1, 28, 28), (10000, 1, 28, 28))

In [42]:
# 网络参数
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)
epoch = 0
network = SimpleConvNet(input_dim=(1, 28, 28), 
                        conv_param={'filter_num': 5, 'filter_size':5, 'pad': 0, 'stride':1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)

# 训练
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 反向传播
    grad = network.gradient(x_batch, t_batch)
    # 更新梯度
    for key in grad.keys():
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(f"epoch {epoch:2} | train_acc: {train_acc:.3f} | test_acc: {test_acc:.3f}")
        epoch += 1

epoch  0 | train_acc: 0.099 | test_acc: 0.098
epoch  1 | train_acc: 0.924 | test_acc: 0.931
epoch  2 | train_acc: 0.956 | test_acc: 0.959
epoch  3 | train_acc: 0.970 | test_acc: 0.969
epoch  4 | train_acc: 0.973 | test_acc: 0.970
epoch  5 | train_acc: 0.978 | test_acc: 0.975
epoch  6 | train_acc: 0.983 | test_acc: 0.978
epoch  7 | train_acc: 0.983 | test_acc: 0.978
epoch  8 | train_acc: 0.987 | test_acc: 0.981
epoch  9 | train_acc: 0.989 | test_acc: 0.982
epoch 10 | train_acc: 0.990 | test_acc: 0.983
epoch 11 | train_acc: 0.991 | test_acc: 0.984
epoch 12 | train_acc: 0.994 | test_acc: 0.985
epoch 13 | train_acc: 0.993 | test_acc: 0.985
epoch 14 | train_acc: 0.993 | test_acc: 0.986
epoch 15 | train_acc: 0.994 | test_acc: 0.986
epoch 16 | train_acc: 0.995 | test_acc: 0.987
